In [80]:
import boto3

# get clients and resources
ec2 = boto3.resource('ec2')
elbv2 = boto3.client('elbv2')
asg = boto3.client('autoscaling')
route53 = boto3.client('route53')

# Let's keep all the names in one place
load_balancer_name = 'application-load-balancer'
instances_ami_name = 'java-application-ami'
instances_security_group_name = 'instances-security-group'
balancer_security_group_name = 'balancer-security-group'
launch_configuration_name = 'asg-launch-configuration'
target_group_name = 'asg-target-group'
auto_scaling_group_name = 'auto-scaling-group'
auto_scaling_policy_name = 'auto-scaling-policy'
domain_name = 'doug-nicholson.net'
balancer_host_name = 'asg.' + domain_name

# Some constants useful for auto-scaling
detailed_monitoring = False
minimum_instance_count = 2
desired_instance_count = 3
maximum_instance_count = 6

In [83]:
# We need to get the id of the image we are going to use.
# We will assume that it already exists.
image_iterator = ec2.images.filter(
    Filters=[
        {
            'Name': 'name',
            'Values': [
                instances_ami_name
            ]
        }
    ]
)
for image in image_iterator:
    if image.name == instances_ami_name:
        image_id = image.image_id
        break
print('AMI found')

AMI found


In [84]:
# Create the security group for the load balancer
balancer_security_group = ec2.create_security_group(
    GroupName=balancer_security_group_name,
    Description='Security group for the load balancer'
)
response = balancer_security_group.authorize_ingress(
    IpPermissions=[
        {
            'FromPort': 80,
            'ToPort': 80,
            'IpProtocol': 'tcp',
            'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
        }
    ]
)
print('balancer security group created')

balancer security group created


In [85]:
# Create the security group for the instances.  Leave the security
# group open from anywhere for testing.
instances_security_group = ec2.create_security_group(
    GroupName=instances_security_group_name,
    Description='Security group for the instances'
)
response = instances_security_group.authorize_ingress(
    IpPermissions=[
        {
            'FromPort': 4567,
            'ToPort': 4567,
            'IpProtocol': 'tcp',
            'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
            'UserIdGroupPairs': [
                {
                    'GroupId': balancer_security_group.group_id,
                    'UserId': balancer_security_group.owner_id
                }
            ]
        }
    ]
)
print('instances security group created')

instances security group created


In [86]:
# Create a launch configuration
response = asg.create_launch_configuration(
    LaunchConfigurationName=launch_configuration_name,
    ImageId=image_id,
    SecurityGroups=[
        instances_security_group_name
    ],
    InstanceType='t2.micro',
    InstanceMonitoring={
        'Enabled': detailed_monitoring
    }
)

In [87]:
# Create the target group for the application load balancer
response = elbv2.create_target_group(
    Name=target_group_name,
    Protocol='HTTP',
    Port=4567,
    VpcId=balancer_security_group.vpc_id,
    HealthCheckProtocol='HTTP',
    HealthCheckPort='4567',
    HealthCheckPath='/health',
    HealthCheckIntervalSeconds=10,
    HealthCheckTimeoutSeconds=5,
    HealthyThresholdCount=5,
    UnhealthyThresholdCount=2,
    TargetType='instance'
)
for target_group in response['TargetGroups']:
    if target_group['TargetGroupName'] == target_group_name:
        target_group_arn = target_group['TargetGroupArn']
        break
print('target group created')

target group created


In [88]:
# Create a list of subnet ids for the load balancer
subnet_iterator = ec2.subnets.filter(
    Filters=[
        {
            'Name': 'vpc-id',
            'Values': [
                balancer_security_group.vpc_id
            ]
        }
    ]
)
subnet_ids = [subnet.subnet_id for subnet in subnet_iterator]
availability_zones = [subnet.availability_zone for subnet in subnet_iterator]

In [89]:
# Create a load balancer
response = elbv2.create_load_balancer(
    Name=load_balancer_name,
    Subnets=subnet_ids,
    SecurityGroups=[
        balancer_security_group.group_id
    ]
)
for load_balancer in response['LoadBalancers']:
    if load_balancer['LoadBalancerName'] == load_balancer_name:
        balancer_dns_name = load_balancer['DNSName']
        balancer_zone_id = load_balancer['CanonicalHostedZoneId']
        load_balancer_arn = load_balancer['LoadBalancerArn']
        break
print('load balancer created')

load balancer created


In [90]:
# Create a listener
response = elbv2.create_listener(
    LoadBalancerArn=load_balancer_arn,
    Protocol='HTTP',
    Port=80,
    DefaultActions=[
        {
            'Type': 'forward',
            'TargetGroupArn': target_group_arn
        }
    ]
)
print('listener created')

listener created


In [91]:
# Now we can create the auto-scaling group
response = asg.create_auto_scaling_group(
    AutoScalingGroupName=auto_scaling_group_name,
    LaunchConfigurationName=launch_configuration_name,
    MinSize=minimum_instance_count,
    MaxSize=maximum_instance_count,
    DesiredCapacity=desired_instance_count,
    AvailabilityZones=availability_zones,
    TargetGroupARNs=[
        target_group_arn
    ],
    HealthCheckType='ELB',
    HealthCheckGracePeriod=180
)

In [92]:
# For some reason accessing the application using the balancer
# host name while the balancer is still provisioning will cause
# problems so wait until the balancer is available.
waiter = elbv2.get_waiter('load_balancer_available')
waiter.wait(
    LoadBalancerArns=[
        load_balancer_arn
    ]
)

In [93]:
# add the public ip address to route53 so SSH is easier
response = route53.list_hosted_zones_by_name(
    DNSName=domain_name
)
local_zone_id = response['HostedZones'][0]['Id'][12:]
response = route53.change_resource_record_sets(
    HostedZoneId=local_zone_id,
    ChangeBatch={
        'Changes': [
            {
                'Action': 'CREATE',
                'ResourceRecordSet': {
                    'Name': balancer_host_name,
                    'Type': 'A',
                    'AliasTarget': {
                        'HostedZoneId': balancer_zone_id,
                        'DNSName': balancer_dns_name,
                        'EvaluateTargetHealth': False
                    }
                }
            }
        ]
    }
)

# it seems like waiting will be the right thing to do
route53_waiter = route53.get_waiter('resource_record_sets_changed')
route53_waiter.wait(
    Id=response['ChangeInfo']['Id'][8:]
)
print('record for ' + balancer_host_name + ' created')

record for asg.doug-nicholson.net created


In [94]:
# Success!
print('launch auto-scaling group script completed')

launch auto-scaling group script completed
